<a href="https://colab.research.google.com/github/patutoes/BERT-Embedding-for-Sentiment-Analysis/blob/main/Sentiment_Analysis_using_BERT_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Importing dependencies

In [2]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 7.9MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=f08a72460e05f755741783c3fccc995729dbff6b1551ec1e8085634e546ad667
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=45db91f5135583a33a2757ad03f0194a54f3c93b3e0f3b3ef97e0a9d4dad57de
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=77df88db1c5954cb91257615cadb0c51f83b9ac4af6f639aa8d8989344749286
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 18.1MB/s 


In [4]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

# Stage 2: Data preprocessing

In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/Project Data/training.1600000.processed.noemoticon.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [11]:
data.tail()

,sentiment,text
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,happy #charitytuesday @theNSPCC @SparksCharity...


In [8]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

## Preprocessing

### Cleaning

In [9]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing user pings
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing URLs
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Removing special characters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [13]:
data_labels = data.sentiment.values
#Changing positive sentiment from 4 to 1
data_labels[data_labels == 4] = 1

### Tokenization

Creating BERT layer to access tokenizer

In [14]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
#Using uncased tokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#For converting to lowercase
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

We are only using the first sentence for BERT inputs so we add the CLS token at the beginning and the SEP token at the end of each sentence.

In [15]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [16]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Dataset creation

We need to create the 3 different inputs for each sentence.

In [17]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    #masking characters after [PAD]
    return np.char.not_equal(tokens, "[PAD]").astype(int)
#Standard function in case we use multiple sentences for each input
def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids

We will create padded batches (so we pad sentences for each batch independently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [18]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
#shuffling data since it's pre-sorted
random.shuffle(data_with_len)
#sorting data by length
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [19]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [20]:
BATCH_SIZE = 32
#getting padded batches
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [21]:
#splitting into train and test set
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [22]:
next(iter(train_dataset))

(<tf.Tensor: shape=(32, 3, 10), dtype=int32, numpy=
 array([[[  101,  1045,  4618,  4339,  2003,  2062,  2411,  1999,  2394,
            102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1,
              1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,
              0]],
 
        [[  101,  4931,  2100,  2017,  7929,  1029,  1029,  1029, 22038,
            102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1,
              1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,
              0]],
 
        [[  101,  1037,  5457, 28616, 28582,  2094,  4013,  2166,  7423,
            102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1,
              1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,
              0]],
 
        [[  101,  1045,  2031,  1037,  2878,  1999,  2026, 10818,  2205,
            102],
         [    1,     1,     1

# Stage 3: Model building

In [23]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
      #ignore first output since we're not using sentence context, only word context
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        print(x.shape)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Stage 4: Training

In [24]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [25]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [26]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [28]:
checkpoint_path = "/content/drive/MyDrive/Project Data/Checkpoints/bert_ckpt_embedding"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Checkpoint Restored")

Checkpoint Restored


In [30]:
class CustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

## Result

In [31]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[CustomCallback()])

Epoch 1/5
(None, None, 768)
(None, None, 768)
40623/40623 [==============================] - 2283s 55ms/step - loss: 0.3238 - accuracy: 0.8615
Checkpoint saved at /content/drive/MyDrive/Project Data/Checkpoints/bert_ckpt_embedding.
Epoch 2/5
40623/40623 [==============================] - 2253s 55ms/step - loss: 0.3170 - accuracy: 0.8645
Checkpoint saved at /content/drive/MyDrive/Project Data/Checkpoints/bert_ckpt_embedding.
Epoch 3/5
40623/40623 [==============================] - 2258s 55ms/step - loss: 0.3112 - accuracy: 0.8670
Checkpoint saved at /content/drive/MyDrive/Project Data/Checkpoints/bert_ckpt_embedding.
Epoch 4/5
40623/40623 [==============================] - 2268s 55ms/step - loss: 0.3055 - accuracy: 0.8689
Checkpoint saved at /content/drive/MyDrive/Project Data/Checkpoints/bert_ckpt_embedding.
Epoch 5/5
40623/40623 [==============================] - 2243s 55ms/step - loss: 0.3009 - accuracy: 0.8704
Checkpoint saved at /content/drive/MyDrive/Project Data/Checkpoints/bert_

# Stage 5: Evaluation

In [32]:
results = Dcnn.evaluate(test_dataset)
print(results)

(None, None, 768)
4513/4513 [==============================] - 105s 23ms/step - loss: 0.3510 - accuracy: 0.8558
[0.35098958015441895, 0.8557708263397217]


In [33]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simulates a batch

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive".format(
            output))

In [34]:
get_prediction("This is a mediocre play for an unintelligent audience.")

(1, 17, 768)
Output of the model: [[0.1147621]]
Predicted sentiment: negative
